# This code runs predictions for external dataset 1 and match the output with ground truth of external dataset

## Import Libraries

In [1]:
import sys
sys.path.insert(0, '../')
#from models.model_mrcnn import _default_mrcnn_config, build_default
import torchvision
import torch
from models.generalized_mask_rcnn_pl import LitMaskRCNN
import torchvision.ops.boxes as bops
import torchvision
from torchvision import transforms
import torch
from PIL import Image
import cv2
import numpy as np
import os
import pandas as pd
import os
import pyvips as Vips
from data.inference_data.Reinhard import Reinhard
from openslide import OpenSlide
from PIL import Image, ImageDraw
from glob import glob
import random
from utils.infer_utils import get_outputs_nms, prepare_input
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

## Load dataset

In [2]:
image_dataset =  "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/1536_tiles"
image_folders =  os.listdir(image_dataset)
image_details_csv =  "/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/Datasets/External_Dataset-Others/gladstone_results/image_details.csv"

In [4]:
coords_data = pd.read_csv(image_details_csv)

In [5]:
train_df = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/classification_dataset/train.csv")

In [6]:
train_df["imagename"] =train_df["imagename"].apply(lambda l: l.split("/")[-1])

In [8]:
merged_data = pd.merge(coords_data,train_df,  on="imagename",how="inner")

In [9]:
# merged_data = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/predictions_ucdavis.csv")

## Load trained model

In [ ]:
model_name = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/runpod_mrcnn_models/yp2mf3i8_epoch=108-step=872.ckpt"
model = LitMaskRCNN.load_from_checkpoint(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval().to(device)

## Preprocess external data input to make it as same format as trained model input

In [11]:
def zoom_and_crop(image_path, crop_coords, zoom_factor, blob_coords):
    """
    Zoom into an image and crop a specific region.
    Parameters:
    - image_path (str): Path to the input image.
    - crop_coordinates (tuple): Coordinates for the crop (x, y, width, height).
    - zoom_factor (float): Factor by which to zoom into the image.
    - output_path (str): Path to save the cropped and zoomed image.
    Returns:
    - None
    """
    # Open the image
    img = Image.open(image_path)
    # Calculate the new dimensions for the zoomed image
    zoomed_width = int(img.width * zoom_factor)
    zoomed_height = int(img.height * zoom_factor)
    # Resize the image (zoom in or out)
    img = img.resize((zoomed_width, zoomed_height), Image.LANCZOS)
    # Crop the zoomed image
    # Adjust crop coordinates to match the zoomed image
    x, y, width, height = crop_coords
    zoomed_x = int(x * zoom_factor)
    zoomed_y = int(y * zoom_factor)
    zoomed_width = int(width * zoom_factor)
    zoomed_height = int(height * zoom_factor)
    
    # crop image
    cropped_img = img.crop((zoomed_x, zoomed_y, zoomed_x + zoomed_width, zoomed_y + zoomed_height))
    
    blob_x, blob_y, blob_w, blob_h = blob_coords

    b_zoomed_x = int( (blob_x * zoom_factor)-zoomed_x)
    b_zoomed_y = int( (blob_y * zoom_factor)-zoomed_y)
    b_zoomed_width = int(blob_w * zoom_factor)
    b_zoomed_height = int(blob_h * zoom_factor)
    #zoomed_blob_coords = [b_zoomed_x,b_zoomed_y,b_zoomed_width,b_zoomed_height]
    scale_x = 1024 / 512
    scale_y = 1024 / 512
    zoomed_blob_coords = [b_zoomed_x*scale_x,   b_zoomed_y*scale_y,  b_zoomed_x*scale_x + b_zoomed_width*scale_x,  b_zoomed_y*scale_y + b_zoomed_height*scale_y]
    cropped_img = cropped_img.resize((1024, 1024), Image.LANCZOS)
    return cropped_img, zoomed_blob_coords


def change_to_numpy_array(input_tensor):
    input_tensor_cpu = input_tensor.squeeze(0).cpu()  # Remove batch dimension and move to CPU if necessary
    # Convert to NumPy array
    numpy_array = input_tensor_cpu.permute(1, 2, 0).numpy()  # Convert to HWC format for visualization
    # Scale back to 0-255 range if needed
    numpy_array = (numpy_array * 255).astype(np.uint8)
    return numpy_array

In [13]:
# Function to initialize normalization based on the reference image
def initialize_normalizer(ref_slide_path):
    """
    Use Reinhard to normalize
    """
    print("Initializing normalization with reference slide:", ref_slide_path)
    ref_image = Vips.Image.new_from_file(ref_slide_path)
    normalizer = Reinhard()
    normalizer.fit(ref_image)
    return normalizer


# Function to normalize a single slide
def normalize_slide(target_image, normalizer):
    """
    Normalize target image with the normalizer
    """
    #print("Normalizing slide:", target_slide_path)
    # Load the target slide
    #target_image = Vips.Image.new_from_file(target_slide_path)
    target_image = Vips.Image.new_from_memory(target_image.data, target_image.shape[1], target_image.shape[0], 3, "uchar")
    # Perform normalization
    normalized_image = normalizer.transform(target_image)
    # Generate the output file path
    #file_name = os.path.basename(target_slide_path).replace(".mrxs", ".tif")
    #output_file_path = os.path.join(output_dir, idx+"_" +filepath.split("/")[-1])
    # Save the normalized image
    #normalized_image.write_to_file(output_file_path)
    return normalized_image

In [14]:
# Path to the reference slide
ref_slide_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/XE07-047_1_AmyB_1/image/XE07-047_1_AmyB_1_8381x_120830y_image.png"
# Initialize the normalizer
normalizer = initialize_normalizer(ref_slide_path)

Initializing normalization with reference slide: /gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/XE07-047_1_AmyB_1/image/XE07-047_1_AmyB_1_8381x_120830y_image.png


In [15]:
merged_data["id1"] = merged_data.index

In [16]:
len(merged_data)

61424

In [17]:
zoom_factor = 2
def run_prediction(row):
    """
    This functions runs on a each dataframe row 
    loads image, preprocesses it and then generate predictions 
    which are returned as a dict labels, matched_labels and scores
    """
    #print(row["id1"], "... of 61424")
    # get image path
    image_path = os.path.join(image_dataset, row["source"], "0",str(row["tile_row"]),str(row["tile_column"])+".jpg")
    
    # crop coords
    crop_coords = row["image coordinates (xywh)"].replace("[","").replace("]","").split()
    crop_coords = [int(x) for x in crop_coords]
    
    #blob coords
    blob_coords = row["blob coordinates (xywh)"].replace("[","").replace("]","").split()
    blob_coords = [int(x) for x in blob_coords]

    # zoom and crop
    cropped_img, zoomed_blob_coords = zoom_and_crop(image_path, crop_coords, zoom_factor,blob_coords)
    
    # normalize image
    norm_img = normalize_slide(np.array(cropped_img), normalizer)
    
    # create input tensor
    input_tensor, image_float_np = prepare_input(np.array(norm_img))
    
    #generate predictions
    masks, boxes, labels, scores = get_outputs_nms(input_tensor, model, 0.6, 0.25)
    cored_pred = 0
    cg_pred = 0
    diffuse_pred = 0
    caa_pred = 0
    label_list= []
    
    # get ground truth label
    cored_gt = row["cored"]
    diffuse_gt = row["diffuse"]
    CAA_gt =  row["CAA"]
    
    
    #zoomed_blob_coords = [260, 88,260+ 92,88+ 68]
    #print(zoomed_blob_coords)
    for j in range(len(boxes)):
        box1 = torch.tensor([zoomed_blob_coords], dtype=torch.float)
        box2 = torch.tensor([boxes[j]], dtype=torch.float)
        iou = bops.box_iou(box1, box2)
        #print(iou)
        if iou>0:
            label_list.append(labels[j])
    return {"all_labels":labels, "matched_labels" :label_list,  "scores":scores}

In [ ]:
# generate predictions for all elements of dataframe 
merged_data["pred_labels"] =merged_data.apply(lambda row:  run_prediction(row), axis=1)

In [138]:
# count all predictions
merged_data["all_pred_count"]  = merged_data["pred_labels"].apply(lambda l: Counter(l["all_labels"]))  
# count all matched labels
merged_data["matched_count"]  = merged_data["pred_labels"].apply(lambda l: Counter(l["matched_labels"]))

# Check class wise match
merged_data["matched_cored_pred"] =merged_data["matched_count"].apply(lambda l: 0 if l.find("Cored")==-1 else 1)
merged_data["matched_diffuse_pred"] =merged_data["matched_count"].apply(lambda l: 0 if l.find("Diffuse")==-1 else 1)
merged_data["matched_cg_pred"] =merged_data["matched_count"].apply(lambda l: 0 if l.find("Coarse-Grained")==-1 else 1)
merged_data["matched_caa_pred"] =merged_data["matched_count"].apply(lambda l: 0 if l.find("CAA")==-1 else 1)

In [106]:
# Keep dataset with atleast 1 match
one_matched = merged_data[ (merged_data["matched_diffuse_pred"]>0) | (merged_data["matched_cg_pred"]>0) | (merged_data["matched_caa_pred"]>0) | (merged_data["matched_cored_pred"]>0)]

In [108]:
# filter neg and notsure
one_matched = one_matched[one_matched["negative"]<1]
one_matched = one_matched[one_matched["notsure"]<=0]

# format variables
one_matched["diffuse_bool"] = one_matched["diffuse_bool"].astype(int)
one_matched["cored_bool"] = one_matched["cored_bool"].astype(int)
one_matched["cored_bool"] = one_matched["cored_bool"].astype(int)

### Precision, Recall, f1 score for Diffuse

In [114]:
confusion_matrix(one_matched["diffuse_bool"], one_matched["matched_diffuse_pred"])
precision_recall_fscore_support(one_matched["diffuse_bool"], one_matched["matched_diffuse_pred"],average='binary')

(0.6148359486447932, 0.7329931972789115, 0.6687354538401862, None)

### Precision, Recall, f1 score for Cored

In [115]:
confusion_matrix(one_matched["cored_bool"], one_matched["matched_cored_pred"])
precision_recall_fscore_support(one_matched["cored_bool"], one_matched["matched_cored_pred"],average='binary')

(0.8425369054127939, 0.19739960289502337, 0.31985885527476515, None)

### Precision, Recall, f1 score for Cored + Coarse-grained

In [120]:
one_matched["matched_cored_cg"] = one_matched.apply(lambda row: max(row["matched_cored_pred"] , row["matched_cg_pred"]), axis=1)
precision_recall_fscore_support(one_matched["cored_bool"], one_matched["matched_cored_cg"], average='binary')

(0.7290297642056436, 0.2415935438416704, 0.36291913214990135, None)

### Precision, Recall, f1 score for CAA

In [121]:

one_matched["CAA_bool"] = one_matched["CAA_bool"].astype(int)
confusion_matrix(one_matched["CAA_bool"], one_matched["matched_caa_pred"])
precision_recall_fscore_support(one_matched["CAA_bool"], one_matched["matched_caa_pred"], average='binary')

(0.3239760699493787, 0.6305418719211823, 0.42802857577139386, None)

In [122]:
# save files
merged_data.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/predictions_ucdavis_final.csv")
one_matched.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/predictions_ucdavis_matched.csv")